<h1><b>Άσκηση 4</b></h1>
<p align="justify">Η μέθοδος Monte Carlo είναι μια υπολογιστική μέθοδος, που βασίζεται στο νόμο των μεγάλων αριθμών. Αν {Χ<sub>n</sub>}<sub>n∈N</sub> είναι μια ακολουθία από ανεξάρτητες, ισόνομες τυχαίες μεταβλητές, με πεπερασμένη μέση τιμή Ε[Χ], τότε:</p>

$$
P\left[
\frac{1}{n}\sum_{k=1}^{n}X_k \rightarrow E[X]
\right] = 1
$$

<p align="justify">Προκειμένου να υπολογίσουμε τη μέση τιμή Ε[Χ]  μιας τυχαίας μεταβλητής Χ, μπορούμε λοιπόν να πάρουμε το μέσο όρο ενός μεγάλου αριθμού ανεξάρτητων δειγμάτων αυτής της μεταβλητής. Με παρόμοιο τρόπο, μπορούμε να προσεγγίσουμε υπολογιστικά την πιθανότητα ενός ενδεχομένου από το κλάσμα των πραγματοποιήσεών του σε μια σειρά από <b>m</b> ανεξάρτητες προσομοιώσεις μέχρι το βήμα <b>n</b>, δηλαδή:</p>

$$
P\left[
\frac{1}{m} \sum_{k=1}^{m}H_k \rightarrow P[X_n | X_0]
\right] = 1
$$

<p align="justify">όπου η τυχαία μεταβλητή Η_k παίρνει την τιμή 1 εάν το ενδεχόμενο πραγματοποιείται στο τέλος του εκάστοτε πειράματος και 0 στην αντίθετη περίπτωση. Σ’ αυτήν την ιδέα θα βασιστεί η άσκηση αυτή. Σας δίνεται η μαρκοβιανή αλυσίδα στο χώρο καταστάσεων <b>Χ</b>={1,2,3} με πίνακα πιθανοτήτων μετάβασης:</p>

$$
P = \begin{pmatrix}
0 & 1 & 0\\
0 & 2/3 & 1/3\\
1/6 & 5/6 & 0
\end{pmatrix}
$$

<p align="justify">Χρησιμοποιώντας το πρόγραμμα που δίνεται παρακάτω, θα πραγματοποιήσετε <b>m</b> ανεξάρτητα πειράματα για να εκτιμήσετε την πιθανότητα <b>Για να τρέξετε το πρόγραμμα θα πρέπει να έχετε φορτώσει το αρχείο <i><a href="https://github.com/nkostopoulos/StochasticsLabPublic/blob/master/lab3/simple_markov_chain_lib.py">simple_markov_chain_lib.py</a></i></b>.</p>

$$
P\left[
X_{40} = 1 | X_0 = 1
\right]
$$

<p align="justify">δηλαδή την πιθανότητα να βρίσκεται η αλυσίδα στην κατάσταση 1 στο 40ό βήμα της δεδομένου ότι ξεκίνησε από την κατάσταση 1. Για να ελέγξετε την ορθότητα της μεθόδου, το πρόγραμμα περιλαμβάνει και τον ακριβή υπολογισμό της παραπάνω πιθανότητας.</p> 
<ul>
<li>Να μελετήσετε το πρόγραμμα και να περιγράψετε σύντομα τη μέθοδο που ακολουθείται.</li>
<li>Να επαναλάβετε τη διαδικασία για τιμές της παραμέτρου m: (α) 1,000, (β) 10,000, (γ) 50,000, (δ) 100,000, (ε) 500,000. Να καταγράψετε και να σχολιάσετε την τιμή της παραπάνω πιθανότητας όπως υπολογίζεται από την προσομοίωση σε σχέση με την ακριβή τιμή της.</li>
</ul>


In [1]:
from __future__ import division
from simple_markov_chain_lib import markov_chain
import statistics as stat
from numpy import matmul
import numpy as np

from time import perf_counter

def defineMarkovTable(): 
    p = 1/6
    markov_table = {
        1: {2: 1.},
        2: {2: 2/3, 3: 1/3},
        3: {1: p, 2: 1-p}
    }

    return markov_table

def defineNumpyTable():
    Pn = np.array([[0,1,0],
           [0,2/3,1/3],
           [1/6, 5/6, 0]])
    P0 = np.array([[1,0,0]])

    return Pn,P0

def multiplyNumpyTables(Pn,P0):
    for index in range(40):
        Pn = np.matmul(Pn,Pn)
    Pn = np.matmul(P0,Pn)
    return Pn

def defineInitDistribution():
    init_dist = {1: 1.}
    
    return init_dist

def calculateProbabilities(markov_table, init_dist, experiments):
    mc = markov_chain(markov_table, init_dist)
    steps = 40
    visits = 0

    for index in range(experiments):
        mc.start()
        for j in range(steps):
            mc.move()
        if mc.running_state == 1: visits += 1

    probability = visits / experiments
    return probability

if __name__ == "__main__":
    markov_table = defineMarkovTable()
    init_dist = defineInitDistribution()

    for m in [1000, 10000, 50000, 100000, 500000]:
        t1_start = perf_counter()
        probability = calculateProbabilities(markov_table, init_dist, m)
        t1_end = perf_counter()
        print("After {:>6} experiments the probability is {} (time needed: {:.4f} seconds)".format(m, probability, t1_end - t1_start))

    Pn,P0 = defineNumpyTable()
    realProbability = multiplyNumpyTables(Pn,P0)
    print("The real probability is:", realProbability[0][0])

After   1000 experiments the probability is 0.039 (time needed: 0.3292 seconds)
After  10000 experiments the probability is 0.0406 (time needed: 0.3664 seconds)
After  50000 experiments the probability is 0.03922 (time needed: 1.7960 seconds)
After 100000 experiments the probability is 0.04064 (time needed: 3.5735 seconds)
After 500000 experiments the probability is 0.039872 (time needed: 17.9731 seconds)
The real probability is: 0.0399985669529409


- Το συγκεκριμένο πρόγραμμα Python που δίνεται στην εκφώνηση, χρησιμοποιεί την υπολογιστική μέθοδο Monte Carlo προκειμένου να προσδιορίσει αριθμητικά την πιθανότητα $\mathbb{P}(X_{40} = 1|X_{0} = 1)$. Κάτι τέτοιο επιτυγχάνεται με διεξαγωγή m ανεξάρτητων πειραμάτων, όπου σε καθένα από αυτά προσομοιάζεται η μαρκοβιανή αλυσίδα της άσκησης για 40 βήματα. Ένα πείραμα θεωρείται επιτυχία, στην περίπτωση όπου έπειτα από 40 βήματα η αλυσίδα βρίσκεται στην κατάσταση 1. Δεδομένου ότι η μέθοδος Monte Carlo βασίζεται στον νόμο των μεγάλων αριθμών, η ζητούμενη πιθανότητα προσδιορίζεται ως το ποσοστό των επιτυχών πειραμάτων, προς το συνόλο των πειραμάτων που εκτελέστηκαν. Ο νόμος των μεγάλων αριθμών μας διαβεβαιώνει ότι καθώς ο αριθμός των πειραμάτων που διεξάγονται τείνει προς το άπειρο, η προσέγγιση που παίρνουμε συγκλίνει στην πραγματική πιθανότητα που αναζητούμε. Η πραγματική τιμή της ζητούμενης πιθανότητας υπολογίζεται ως εξής:
\begin{equation}
    \begin{aligned}
        \mathbb{P}(X_{40} = 1|X_{0} = 1) = (\pi_{0}P^{40})_{1} = ([1, 0, 0]\text{ }P^{40})_{1} = 0.0399985669529409,
    \end{aligned}
\end{equation}
όπου με $(\cdot)_{i}$ υποδηλώνουμε την i-οστή συνιστώσα του διανύσματος $(\cdot)$.

- Από την παραπάνω έξοδο του κώδικα παρατηρούμε ότι ακόμη και με 1000 επαναλήψεις έχουμε ακρίβεια τριών δεκαδικών ψηφίων. Γενικά, έπειτα από αρκετές εκτελέσεις του συγκεκριμένου προγράμματος παρατηρούμε ότι για m = 50000 ή 100000 προκύπτουν πολύ αξιόπιστα αποτελέσματα, με αισθητά λιγότερη απαίτηση χρόνου για την ολοκλήρωση της προσομοίωσης συγκριτικά με τα 500000 πειράματα. Προφανώς, καθώς το πλήθος m των πειρμάτων που διεξάγονται αυξάνεται, το απόλυτο σφάλμα μεταξύ της πραγματικής πιθανότητας και αυτής που υπολογίζουμε, μειώνεται. Ωστόσο, παρατηρούμε σημαντική αύξηση στον υπολογιστικό χρόνο που απαιτείται. Μάλιστα, η σύγκλιση επιτυγχάνεται πολύ καλά στα 50000 πειράματα, οπότε αυτή την τιμή θα επιλέγαμε για να ικανοποιήσουμε το trade-off μεταξύ ακρίβειας και υπολογιστικού χρόνου.